# O que o twitter está pensando?

# Extraindo informações em redes sociais utilizando Python

`> por: `[@profadolfoguimaraes](http://www.twitter.com/profadolfoguimaraes)

Estes tutoriais apresentam os principais scritps desenvolvidos no minicurso: **O que o twitter está pensando? Extraindo informações em redes sociais utilizando Python**. O conteúdo está dividio em dois repositórios: (1) [d2l-minicursotwitter-notebook](http://github.com/adolfoguimaraes/d2l-minicursotwitter-notebook) que possui estes notebooks e (2) [d2l-minicursotwitter-web](http://github.com/adolfoguimaraes/d2l-minicursotwitter-web) que possui a página web desenvolvida.

O material completo do minicurso pode ser encontrado em: http://www.data2learning.com/cursos.

## 04 - Juntando tudo

Ao longo deste tutorial aprendemos como selecionar dados do twitter e fazer algumas etapas de pré-processamento. O objetivo desta seção é juntar tudo. Vamos coletar dados do twitter, pré-processar e imprimir a lista de tokens mais frequentes para um conjunto de dados coletados.

* O primeiro passo é coletar tweets e adicionar em uma lista. Para nossa tarefa só interessa o texto e o login do usuário que está postando. Demais informações podem ser descartadas.
* Em seguida vamos processar os textos coletados tokenizando.
* Deve ser armazenado uma lista das hashtags mais postadas e dos usuários que mais postaram.
* Em seguida devemos eliminar as stopwords.
* Devemos eliminar eventuais problemas no texto: palavras repetidas e escritas de forma incorreta.
* Por fim, extrair os 50 termos mais frequentes usando bigram e trigram.
* Ao final deve imprimir a lista dos termos, a lista das hashtags e a lista dos usuários.

In [1]:
# Importando as bibliotecas necessárias

from twython import Twython
from datetime import datetime

#Definição das chaves da API do Twitter
consumer_key = None # Get Keys and Access Token at apps.twitter.com
consumer_secret = None # Get Keys and Access Token at apps.twitter.com
access_token = None # Get Keys and Access Token at apps.twitter.com
access_token_secret = None # Get Keys and Access Token at apps.twitter.com

tw = Twython(consumer_key, consumer_secret, access_token, access_token_secret)

result = tw.search(q="vigadores",count=100,lang="pt")

tweets = result['statuses']

list_text = []
list_user = []

for tweet in tweets:
    
    #Evita pegar texto truncado no caso de Retweeteds
    if 'retweeted_status' in tweet.keys():
        list_text.append(tweet['retweeted_status']['text'])
    else:
        list_text.append(tweet['text'])
        
    list_user.append(tweet['user']['screen_name'])    

As variáveis **list_text** e **list_user** armazenam a lista de todos os textos e usuários, respectivamente. O passo seguinte é tratar essas listas com as etapas de pré-processamento.

Vamos tratar todos os tweets como um único texto e fazer o pré-processamento nesse texto. A partir do texto vamos extrair a lista de palavras mais citadas, lista de usuários citados e lista de hashtags. 

In [2]:
from unicodedata import normalize, category
from nltk.tokenize import regexp_tokenize
from collections import Counter, Set
from nltk.corpus import stopwords


pattern = r'(https://[^"\' ]+|www.[^"\' ]+|http://[^"\' ]+|\w+|\@\w+|\#\w+)'
portuguese_stops = stopwords.words(['portuguese'])

users_cited = []
links_appears = []
hashtags = []

patterns = []

for text in list_text:
    
    text = text.lower()
    
    patterns += regexp_tokenize(text, pattern)

    users_cited += [e for e in patterns if e[0] == '@']
    links_appears += [e for e in patterns if e[:4] == 'http']
    hashtags += [e for e in patterns if e[0] == '#']
    
    final_tokens = [e for e in patterns if e[:4] != 'http']
    final_tokens = [e for e in final_tokens if e[:4] != 'www.']
    final_tokens = [e for e in final_tokens if e[0] != '#']
    final_tokens = [e for e in final_tokens if e[0] != '@']
    
    
words = [word for word in final_tokens if word not in portuguese_stops]

word_set = set(words)

In [3]:
#Classes para correção
import re

class RepeatReplacer(object):
    
    def __init__(self):
        self.repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
        self.repl = r'\1\2\3'
        
    def replace(self, word):
        repl_word = self.repeat_regexp.sub(self.repl, word)

        if repl_word != word:
            return self.replace(repl_word)
        else:
            return repl_word


In [4]:
#Cria um map das palavras erradas para as palavras corrigidas

replacer_repeat = RepeatReplacer()

new_words = []
map_words = {}
for word in word_set:
    
    new_word = replacer_repeat.replace(word)
    
    map_words[word] = new_word

In [5]:
words_temp = [map_words[word] for word in words ]
words_temp = [word for word in words_temp if len(word) >= 3]

final_words = []

for word in words_temp:
    try:
        new_word = normalize('NFKD', word.lower()).encode('ASCII','ignore')
    except UnicodeEncodeError:
        new_word = normalize('NFKD', word.lower().decode('utf-8')).encode('ASCII','ignore')

    final_words.append(new_word.decode("utf-8"))

In [6]:
import nltk

frequence_terms = nltk.FreqDist(final_words)

for word in frequence_terms.most_common(10):
    print(word[0])

vigadores
guera
infinita
ver
filme
pra
asistir
sobre
spoiler
asisti


In [7]:
# Retornando os usuários, usuários citados e hashtags mais frequentes
frequence_users = nltk.FreqDist(list_user)
frequence_users_cited = nltk.FreqDist(users_cited)
frequence_hashtags = nltk.FreqDist(hashtags)

print(frequence_users.most_common(10))
print(frequence_users_cited.most_common(10))
print(frequence_hashtags.most_common(10))

[('rmadridybala', 2), ('JessiFeijoli', 2), ('Carol_Castro_B', 2), ('medicilouis', 2), ('WiliamCruzdaSi1', 2), ('narryhelike', 1), ('madorattioto', 1), ('Carolll1906', 1), ('Crystopher_Kau', 1), ('SamOdev', 1)]
[('@anatuzino_', 90), ('@twitter', 85), ('@n3rdizito', 82), ('@leticiaa_rl', 61), ('@crfnicole', 52), ('@legadodacopa', 40), ('@peterjordan100', 39), ('@nathalialivya', 37), ('@marcioroocha', 35), ('@renatoestranho', 25)]
[('#quintadetremurasdv', 94), ('#vigadores', 85), ('#guerrainfinita', 85), ('#quartadetremurasdv', 65), ('#diadedecepcao', 26), ('#fanaccountsdv', 7)]


In [8]:
#Pegando os bigram e trigram mais frequentes

from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures

bcf = BigramCollocationFinder.from_words(final_words)
tcf = TrigramCollocationFinder.from_words(final_words)

bcf.apply_freq_filter(3)
tcf.apply_freq_filter(3)

result_bi = bcf.nbest(BigramAssocMeasures.raw_freq, 5)
result_tri = tcf.nbest(TrigramAssocMeasures.raw_freq, 5)


for r in result_bi:
    print(r)
print
for r in result_tri:
    print(r)


('vigadores', 'guera')
('guera', 'infinita')
('ver', 'vigadores')
('asistir', 'vigadores')
('sobre', 'vigadores')
('vigadores', 'guera', 'infinita')
('sobre', 'vigadores', 'guera')
('aguento', 'ver', 'coisas')
('coisas', 'sobre', 'vigadores')
('guera', 'infinita', 'chega')


Com isso finalizamos nossa parte de pré-processamento de texto. Observe que só extraindo de forma correta os termos mais frequentes temos uma visão melhor do que está sendo discutido nas redes sociais. No entanto, exibir isso somente na linha de comando não é muito atrativo. O próximo passo é exibir estas informações em uma página web.

Os arquivos da página web criada podem ser acessados no repositório: [d2l-minicursotwitter-web](http://www.github.com/adolfoguimaraes/d2l-minicursotwitter-web).